Google Earth Engine ingestion workflow for PSScene4Band and PSScene items
=========================================================================

# CLI Tool

PSScene4Band items can be ordered and delivered directly to Google Earth Engine via the CLI tool that comes packaged with the `planet` python library.

In a terminal window (with your python virtual environment activated, if necessary), run `planet --help` to view the documentation for the CLI tool. The result should look something like the following.

```
$ planet --help
Usage: planet [OPTIONS] COMMAND [ARGS]...

  Planet API Client

Options:
  -w, --workers INTEGER  The number of concurrent downloads when requesting
                         multiple scenes. - Default 4

  -v, --verbose          Specify verbosity
  -k, --api-key TEXT     Valid API key - or via ENV variable PL_API_KEY
  -u, --base-url TEXT    Change the base Planet API URL or ENV PL_API_BASE_URL
                         - Default https://api.planet.com/

  --version              Show the version and exit.
  --help                 Show this message and exit.

Commands:
  analytics  Commands for interacting with the Analytics API
  data       Commands for interacting with the Data API
  help       Get command help
  init       Login using email/password
  mosaics    Commands for interacting with the Mosaics API
  orders     Commands for interacting with the Orders API
```

## Basic orders

We're going to focus primarily on the `orders` functionality, starting with order creation.
```
$ planet orders create --help
Usage: planet orders create [OPTIONS]

  Create an order

Options:
  -pp, --pretty / -r, --no-pretty
                                  Format JSON output
  --item-type ITEM-TYPE           Specify an item type  [required]
  --bundle BUNDLE                 Specify bundle  [required]
  --tools FILE                    Path to toolchain json
  --cloudconfig FILE              Path to cloud delivery config
  --email                         Send email notification when Order is
                                  complete

  --clip CLIP                     Provide a GeoJSON AOI Geometry for clipping
  --ids_from_search TEXT          Embedded data search
  --id TEXT                       One or more comma-separated item IDs
  --name TEXT                     [required]
  --help                          Show this message and exit.
```

The simplest use-case for this command is creating an order from a known scene ID or a _list_ of known scene IDs.

This command is all one line

```
planet orders create --item-type psscene4band --bundle analytic_sr_udm2 --id 20211201_183910_1026,20211201_183909_1026,20211201_183907_1026 --name example_order --email
```

Documentation about bundle types can be found here:

https://developers.planet.com/docs/orders/product-bundles-reference/

## GEE delivery

The `--cloudconfig` option can be used to deliver orders directly to Google Earth Engine.


Before placing orders to be delivered ot GEE, you must setup access to your GEE project for Planet's delivery system. [These docs describe the process.](https://developers.planet.com/docs/integrations/gee/quickstart/) If you have an existing GEE project that you plan to use, skip to step 3.

This option requires an additional json file containing the name and credentials for the GEE project. The json file should be structured like so.
```
{
    "google_earth_engine": {
        "project": "your_cloud_project_name",
        "collection": "your_ee_image_collection_name"
    }
}
```

Note the `collection` parameter. You will need to create an empty ImageCollection that the orders system can deliver to.
![Create new ImageCollection](newimgcolection.gif)

The json for the above example would look like this (including the mispelling error on my part).
```
{
    "google_earth_engine": {
        "project": "pre-sales-demos-313313",
        "collection": "MyColection"
    }
}
```



## Caveats

There are two important caveats to bear in mind when ordering imagery to be delivered to GEE:
- Only certain item types are currently supported, and the `PSScene` item type (the only type with 8-band imagery) is _not_ among them. https://developers.planet.com/docs/integrations/gee/delivery/#supported-item-asset-types
- Clipping is the only built-in geoprocessing tool supported for orders being delivered to GEE. https://developers.planet.com/docs/integrations/gee/delivery/#supported-raster-operations

To work around these, we can order imagery to be delivered to a Google Cloud Storage bucket, and then ingest the imagery into GEE from there. The latter part of the next section has an example of that workflow.

# Python script

# Setup

In [ ]:
from planet import api
import json
from pprint import pprint
import arrow

client = api.ClientV1()
# Or, if $PL_API_KEY environment variable is NOT set
#client = api.ClientV1(api_key='API_KEY_HERE')

## Direct GEE delivery
The CLI method above can also be done using the `Planet` python library.

The order from the previous example would be represented in json like so.

```
{
   "name":"example_order",
   "products":[  
      {  
         "item_ids":[
             "20211201_183910_1026",
             "20211201_183909_1026",
             "20211201_183907_1026"
         ],
         "item_type":"PSScene4Band",
         "product_bundle":"analytic_sr_udm2"
      }
    ],
    "delivery": {
        "google_earth_engine": {
            "project": "pre-sales-demos-313313",
            "collection": "MyColection"
        }
    }
}
```

and would be constructed in Python like so

In [ ]:
order_name = "example_order"
scene_ids = [
             "20211201_183910_1026",
             "20211201_183909_1026",
             "20211201_183907_1026"
]
scene_filter = api.filters.string_filter('id',*scene_ids)

item_type = 'PSScene4Band'
bundle_type = 'analytic_sr_udm2'
project_name = "pre-sales-demos-313313"
collection_name = "MyColection"

direct_gee_order = {
   "name": order_name,
   "products":[  
      {  
         "item_ids": scene_ids,
         "item_type": item_type,
         "product_bundle": bundle_type
      }
    ],
    "delivery": {
        "google_earth_engine": {
            "project": project_name,
            "collection": collection_name
        }
    }
}

direct_order_results = client.create_order(direct_gee_order)

pprint(direct_order_results.get())

It can take some time for the order to complete, especially if any of the imagery is relatively old and/or if there are _lots_ of scenes in the order.

You can check on the status of the order like so.

In [ ]:
direct_order_id = direct_order_results.get()['id']
direct_order_status = client.get_individual_order(direct_order_id)
pprint(direct_order_status.get())

GEE ingestion of data delivered to a GCS bucket.

This is specificially an example of ordering 8-band PSScene data to be delivered to a GCS bucket to be ingested into GEE. The same general workflow can be used for data of any item type. See here for more information aboutn configuring your GCS bucket for delivery. https://developers.planet.com/docs/orders/delivery/#delivery-to-google-cloud-storage

# Utility functions

In [ ]:
def list_scenes(geom, toi_start=None, toi_end=None, cloud_pct=50.0, sat_type="PSB.SD"):
    """Accepts: 
       - AOI (geojson geometry string)
       - TOI beginning (ISO format)
       - TOI end (ISO format)
       - Cloud cover percentage (float)
       - Instrument type (string)

       Generates filters, searches for scenes.
       
       Returns list of scenes.
    """
    filters = []
    filters.append(api.filters.geom_filter(geom))
    if toi_start:
        filters.append(api.filters.date_range('acquired',gte=toi_start))
    if toi_end:
        filters.append(api.filters.date_range('acquired',lte=toi_end))
    filters.append(api.filters.range_filter('cloud_percent', lt=50.0))
    filters.append(api.filters.string_filter('instrument', 'PSB.SD'))
    all_filts = api.filters.and_filter(*filters)
    request = api.filters.build_search_request(all_filts,item_types=['PSScene'])
    results = client.quick_search(request)
    scenes = []
    for page in results.iter():
        for item in page.items_iter(limit=None):
            scenes.append(item)
    return scenes

In [ ]:
def order_scenes(scenes, order_name, geom=None, bucket, creds):
    """
        Accepts:
        - List of scene IDs (list of strings)
        - Order name (string)
        - (Optional) AOI for clipping (geojson geometry string)
        - GCS bucket name (string)
        - GCS bucket credentials (string)
            Credentials must be converted to base64 first. See here:
            https://developers.planet.com/docs/orders/delivery/#delivery-to-google-cloud-storage

        Returns a client order object
    """

    order_json = {  
        "name": order_name,
        "order_type": "partial",
        "products":[
            {  
                "item_ids": scenes,
                "item_type": 'PSScene',
                "product_bundle": 'analytic_8b_sr_udm2'
            }
        ],
        "delivery": {
            "google_cloud_storage": {
                "bucket": bucket,
                "credentials": creds
                
            }
        },
        "notifications":{
            "email":True
        }
    }
    if geom:
        order_json['tools'] = [
            {
                "clip": {
                    "aoi": geom
                }
            }
        ]

    #pprint(order_json)
    results = client.create_order(order_json)
    return results

Edit `base_name`  below with the details of your target GEE project and asset.

In [ ]:
def get_scene(scene_id):
  """
    Accepts a single scene ID string.
    Returns scene metadata.
  """
  request = api.filters.build_search_request(api.filters.string_filter('id',scene_id),item_types=['PSScene'])
  result = client.quick_search(request)
  items = []
  for page in result.iter():
      for item in page.items_iter(limit=None):
          items.append(item)
  return items[0]
    
def scene_manifest(scene_id, order_id, bucket):
  """
    Accepts:
    - A single scene ID (string)
    - Order ID (string)
    - GCS bucket (string)

    Returns string containing a GEE upload manifest.
  """
  #base_name = "projects/earthengine-legacy/assets/projects/pre-sales-demos-313313/assets/"
  base_name = "projects/GEE_PROJECT_NAME/assets/GEE_ASSET_NAME/"
  scene = get_scene(scene_id)
  scene['properties']['ground_control'] = str(scene['properties']['ground_control'])
  manifest_name = base_name + "{}".format(scene_id)
  manifest = {
      "name": manifest_name,
      "tilesets": [
          {
              "id": "data_tileset",
              "sources": [
              {
                "uris": [
                  "gs://{}/{}/PSScene/{}_3B_AnalyticMS_SR_8b_clip.tif".format(bucket, order_id, scene_id)
                  #"gs://{}/{}/PSScene/{}_3B_AnalyticMS_8b.tif".format(bucket, order_id, scene_id)
                ]
              }
              ]
          },
          {
              "id": "udm_tileset",
              "sources": [
              {
                "uris": [
                  "gs://{}/{}/PSScene/{}_3B_udm2_clip.tif".format(bucket, order_id, scene_id)
                ]
              }
              ]
          }
      ],
      "properties": scene['properties'],
      "start_time": scene['properties']['acquired'],
      "end_time": scene['properties']['acquired']
  }
  manifest['properties']['id'] = scene_id
  return manifest

# Actual workflow

In [ ]:
# Set up order delivery

# Change values to suit
order_name = 'DixieFire'
bucket_name = "pfed-gee-ingest"

# Build order over AOIs
with open("dixieExtents.geojson") as filein:
    aoi_features = json.load(filein)['features']

# There are multiple AOIs in the layer. Each contains an end date. Iterate over 
# the AOIs, pull TOI from feature. Create order. Store order objects to 
# order_results.
order_results = {}

for i in range(len(aoi_features)):
    geom = aoi_features[i]['geometry']
    end_date = aoi_features[i]['properties']['EndDate'].replace('/','-')
    if i == 0:
        start_date = '2021-07-10'
    else:
        start_date = aoi_features[i-1]['properties']['EndDate'].replace('/','-')
    
    scenes = list_scenes(geom, start_date, end_date)
    scene_ids = [scene['id'] for scene in scenes]
    sub_order_name = order_name + end_date
    order_result = order_scenes(scene_ids,sub_order_name,bucket_name)
    order_results[order_result.get()['id']] = {
        "name": sub_order_name,
        "scenes": scenes,
        "result": order_result,
        "start": start_date,
        "end": end_date
        
    }

In [ ]:
# Check on status of all orders.
for key in order_results.keys():
    status = client.get_individual_order(order_results[key]['result'].get()['id']).get()
    pprint(status['id'])
    pprint(status['state'])
    pprint(status['last_message'])
    #pprint(status)

In [ ]:
# Generate earthengine upload manifests for each scene
for key in order_results.keys():
    for scene in order_results[key]['result'].get()['products'][0]['item_ids']:
        manifest = scene_manifest(scene,order_name,key,bucket_name)
        
        with open('manifests/{}.json'.format(scene),'w') as fileout:
            json.dump(manifest,fileout)

In [ ]:
# Create bash script to run all generated manifest uploads.
# May need to set execution permissions before running script. e.g.:
#
# > chmod +x MyManifestScript.sh
# > ./MyManifestScript.sh
for key in order_results.keys():
    with open("{}.sh".format(key),'w') as filein:
        filein.write("#!/bin/bash\n")
        for scene in order_results[key]['result'].get()['products'][0]['item_ids']:
            upcom = "earthengine upload image --manifest manifests/{}.json\n".format(
                    scene)
            filein.write(upcom)